# Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

#  Widgets

In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("catalogo", "chocolate_sales")
dbutils.widgets.text("esquema_source", "bronze")
dbutils.widgets.text("esquema_sink", "silver")

# Parameters

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

# Read source tables

In [0]:
df_sales = spark.table(f"{catalogo}.{esquema_source}.sales")
df_master_salesman = spark.table(f"{catalogo}.{esquema_source}.master_salesman")
df_master_country = spark.table(f"{catalogo}.{esquema_source}.master_country").withColumnRenamed("country_id","country_id_master")
df_master_product = spark.table(f"{catalogo}.{esquema_source}.master_product").withColumnRenamed("product_id","upc_code")


# Joins

In [0]:
df_join = df_sales.join(df_master_salesman, df_sales.sales_person_id == df_master_salesman.salesman_id, "inner").join(df_master_country, df_sales.country_id == df_master_country.country_id_master, "inner").join(df_master_product, df_sales.product_id == df_master_product.upc_code, "inner")

# Dataframe with selected columns

In [0]:
df_selected = df_join.select("salesman_id","salesman_name","country_id","country", "region_id","region","upc_code","product_desc", "date", "amount", "boxes_shipped")
                             

# Save in table

In [0]:
df_selected.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.sales_final")